In [1]:
from mmnl import MMNL
import numpy as np
import pickle
import pandas as pd


In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

Authorize Google Drive to upload files to the drive

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [2]:
def load_data(path):
    dat = np.load(path)
    X = dat[:, :-1]
    Y = np.reshape(dat[:, -1], (-1, 1))
    return X, Y
X, Y = load_data('data/data.npy')

Load data

In [3]:
%%time
infile = open('./data/500_MC_dgp_uts.p', 'rb')
big_dict = pickle.load(infile)
Y_dgp = big_dict['theta: [ 1.5  1.  -1.1  0.4  0.1  0.6]']
print(Y_dgp.shape)

(2798, 500)
Wall time: 20 ms


In [6]:
%%time
result = []
draws = 250
start = 80
end = 100
method = 'SMC'
for i in range(start,end):
    print('iteration: %i' %(i+1))
    qmc = MMNL(X,Y_dgp[:,i],draws,3,method=method)
    solution = qmc.solver()
    result.append(solution)
    if i%10==9:
        pickle.dump( result, open( "./resultaten/%s(%i)/%i-%i_%s(%i)_utsgdp_results.p" %(method,draws,i-9,i,method,draws), "wb" ) )
        result = []
#     f = drive.CreateFile()
#     f.SetContentFile("./resultaten/%i-%i_%s(%i)_results.p" %(i,i+10,method,draws))
#     f.Upload()

iteration: 81
      fun: 2781.405996982095
 hess_inv: array([[ 1.81062317e-04, -2.06274904e-05, -3.60702496e-04,
        -1.01026775e-03, -1.41323117e-03, -1.79227497e-03],
       [-2.06274904e-05,  4.47195150e-06,  4.35927205e-05,
         1.41546713e-04,  1.99432358e-04,  2.41043062e-04],
       [-3.60702496e-04,  4.35927206e-05,  7.31111261e-04,
         2.09177557e-03,  2.92828258e-03,  3.68668742e-03],
       [-1.01026775e-03,  1.41546713e-04,  2.09177557e-03,
         6.23692360e-03,  8.74757338e-03,  1.08632344e-02],
       [-1.41323116e-03,  1.99432356e-04,  2.92828256e-03,
         8.74757333e-03,  2.43881316e-01,  4.52836830e-02],
       [-1.79227497e-03,  2.41043064e-04,  3.68668743e-03,
         1.08632345e-02,  4.52836830e-02,  4.15746690e-02]])
      jac: array([ 2.74658203e-04,  3.66210938e-04,  2.74658203e-04, -3.05175781e-05,
        0.00000000e+00,  0.00000000e+00])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 1449
      nit: 3

C:\Users\niels\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:83: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


      fun: nan
 hess_inv: array([[ 2.19899161e-02, -1.90579555e-02,  5.87246505e-03,
         1.10530234e+00,  6.92388612e+00,  1.94833770e+00],
       [-1.90579555e-02,  3.00286568e-02, -6.92410127e-03,
        -1.51622197e+00, -9.08052624e+00, -2.60149250e+00],
       [ 5.87246505e-03, -6.92410127e-03,  3.14353139e-03,
         4.45346397e-01,  2.75466349e+00,  7.77617202e-01],
       [ 1.10530234e+00, -1.51622197e+00,  4.45346397e-01,
         1.49510142e+02,  5.58843274e+02,  2.01970839e+02],
       [ 6.92388612e+00, -9.08052624e+00,  2.75466349e+00,
         5.58843274e+02,  3.18202714e+03,  9.31941108e+02],
       [ 1.94833770e+00, -2.60149250e+00,  7.77617202e-01,
         2.01970839e+02,  9.31941108e+02,  3.01545471e+02]])
      jac: array([nan, nan, nan, nan, nan, nan])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 1320
      nit: 31
     njev: 165
   status: 2
  success: False
        x: array([ 3.01958041e+01, -3.39735693e+01,  1.03610

In [11]:
pickle.dump( result, open( "./resultaten/BQMC(8)/90_BQMC(8)_results.p", "wb" ) )

In [10]:
x_est = np.array(result)
x_est.shape

(100, 6)

In [11]:
np.mean(x_est, axis=0)

array([ 4.58777441,  2.65487859, -7.12572568,  7.45172249,  2.22983804,
        5.23629935])